In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
sys.path.append('/home/jma819/post_cmfe_analysis')
#sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf

/home/jma819/.conda/envs/caiman/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.incremental_pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)


In [107]:
reload(utils_jjm)

<module 'python_utils_jjm' from '/home/jma819/post_cmfe_analysis/python_utils_jjm.py'>

In [31]:
#'/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
#cnmfe_path = '/volumes/My_Passport/cnmfe_analysis_files/batch_output_files/08-Apr_16_03_01_out.mat'
CNMFE_file = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/rotarod/17-Apr_06_31_26_out.mat'

In [32]:
CNMFE_results = sio.loadmat(CNMFE_file)
#np.shape(CNMFE_results['C'])
file_array = CNMFE_results['P'][0][0][13]

In [41]:
#file_array

In [37]:
file_array[0][0][0].split('/')[8]

'H15_M38_S19'

In [39]:
frames = file_array[session][0][0].split('/')[10]
frames

'frames_2969_3958'

In [17]:
# get fluorescence manipulations for whole session
cell_fluorescence = sio.loadmat(CNMFE_file)
C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6)
cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components)

#filter out small cells for analysis
C_normalized_z_scored_filtered = utils_jjm.filter_out_by_size(C_normalized_z_scored, cell_contours, 
                                                                  for_dims, 0.6, 100).drop(['msCamFrame'], axis=1)

In [19]:
# align frame inidcies in output file to session 
CNMFE_results = sio.loadmat(CNMFE_file)
#np.shape(CNMFE_results['C'])
file_array = CNMFE_results['P'][0][0][13]
session_names = []
frames_indicies = []
for session in range(len(file_array)):
    msCam_session = file_array[session][0][0].split('/')[8]
    frames = file_array[session][0][0].split('/')[10]
    frames_idx = frames.strip('frames_').split('_')
    session_names.append(msCam_session)
    frames_indicies.append(frames_idx)

total_frames = np.sum(np.array([(1+int(frames_indicies[session_idx][1]))-int(frames_indicies[session_idx][0]) for session_idx in range(len(session_names))]))

session_key = pd.DataFrame(frames_indicies, index=session_names)
session_key['cumulative_frames'] = np.nan*np.ones(len(session_names))
session_key = session_key.transpose()

final_frames = pd.DataFrame({k:v for k,v in zip(np.unique(np.array(session_names)), 
                                                [session_key[session].loc[1].values[-1:] for session in np.unique(np.array(session_names))])}, 
                                                index=['final_frame']).transpose()

final_frames['cumulative_frames'] = np.cumsum(np.array([int(frame) for frame in final_frames['final_frame'].values]))

#final_frames

# pull out data by session
session = 'H19_M46_S0'
frame_indicies = []
for session in np.array(final_frames.index):
    session_idx = np.where(np.array(final_frames.index)==session)[0][0]
    frame_end = final_frames.iloc[session_idx]['cumulative_frames']
    if session_idx != 0:
        frame_start = final_frames.iloc[session_idx-1]['cumulative_frames']
    else:
        frame_start=0
    frames=(frame_start, frame_end)
    #start/stop indicies of session
    frame_indicies.append(frames)

final_frames['frame_indicies'] = frame_indicies


AttributeError: 'str' object has no attribute 'values'

In [45]:
final_frames

,final_frame,cumulative_frames,frame_indicies
H19_M46_S0,1977,1977,"(0, 1977)"
H19_M51_S4,2561,4538,"(1977, 4538)"
H19_M55_S57,3388,7926,"(4538, 7926)"
H20_M0_S57,4712,12638,"(7926, 12638)"
H20_M11_S42,5216,17854,"(12638, 17854)"
H20_M17_S28,4574,22428,"(17854, 22428)"
H20_M22_S56,4993,27421,"(22428, 27421)"
H20_M28_S4,4370,31791,"(27421, 31791)"
H20_M33_S18,5993,37784,"(31791, 37784)"
H20_M6_S41,4465,42249,"(37784, 42249)"


In [20]:
C_normalized_z_scored_filtered.head()

,2,3,4,5,8,10,11,12,13,14,...,129,141,146,150,153,160,162,164,165,167
00:00:00,-0.062093,-0.075735,0.002018,-0.066328,-0.02038,0.683446,-0.44541,0.409262,0.170117,0.354901,...,-0.012457,0.216574,0.005305,0.222268,0.102844,-0.379101,0.397314,-0.239740,1.996984,2.981850
00:00:00.050000,-0.062093,-0.120887,0.002498,-0.015717,-0.02038,0.618526,-0.44541,0.291741,0.136495,0.233055,...,-0.012457,0.170167,-0.006043,0.424130,0.417616,2.595728,0.206385,1.360460,1.717187,2.263672
00:00:00.100000,-0.037017,-0.160298,0.002702,-0.019679,-0.02038,0.557987,-0.44541,0.755857,0.106372,0.127952,...,-0.012457,0.519416,-0.013794,0.578982,0.526216,6.586304,0.052788,1.925199,3.224101,4.699146
00:00:00.150000,-0.037829,-0.194698,0.005558,-0.023432,-0.02038,0.501535,-0.44541,0.600847,0.079382,0.037292,...,-0.012457,0.449614,-0.015170,0.549297,0.461904,5.692545,-0.070778,3.013615,2.204221,3.650809
00:00:00.200000,-0.038615,-0.224725,0.005136,-0.026986,-0.02038,0.448892,-0.44541,1.037436,0.055201,-0.040909,...,-0.012457,0.385204,-0.020029,0.400313,0.404659,4.930103,-0.170183,2.246238,1.449031,2.804020


In [30]:
#plt.plot(C_normalized_z_scored[40].values[0:1000])

In [21]:
#plt.plot(C_normalized_z_scored[1].values[frames[0]:frames[1]])

TypeError: slice indices must be integers or None or have an __index__ method

In [60]:
np.array(C_normalized_z_scored_filtered.columns)

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 24, 25, 26, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
       42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
       60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 95, 96, 97, 98,
       99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141,
       142, 145, 148, 149], dtype=object)

In [132]:
by_trial = {}
trial_stats = {}
for session in np.array(final_frames.index):
    event_indicies, event_times, event_ISIs = utils_jjm.get_ISIs(C_normalized_z_scored_filtered.iloc[final_frames.loc[session]['frame_indicies'][0]:final_frames.loc[session]['frame_indicies'][1]], 20, .1, 
                                                                 np.array(C_normalized_z_scored_filtered.columns), 2)
    by_trial[session] = {'event_indicies' : event_indicies, 'event_times': event_times, 'event_ISIs': event_ISIs}
    
    event_counts_by_cell = [len(by_trial['H19_M46_S0']['event_indicies'][array]) for array in range(len(by_trial['H19_M46_S0']['event_indicies']))]
    event_rates_by_cell = [count/((1/20)*float(final_frames.loc[session]['frame_indicies'][1]-final_frames.loc[session]['frame_indicies'][0])) for count in event_counts_by_cell]
    trial_stats[session] = {'event_counts_by_cell' : event_counts_by_cell , 'event_rates_by_cell' : event_rates_by_cell}

In [135]:
np.mean(trial_stats['H19_M46_S0']['event_rates_by_cell'])

0.1323683903350064

In [136]:
np.mean(trial_stats['H20_M6_S41']['event_rates_by_cell'])

0.058609699371177534